In [1]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter
import re
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings("ignore")  # Suppress all warnings

#Reemplazar end point
pd.options.display.max_rows = 20
DB_USERNAME = ''
DB_PASSWORD = ''
DB_HOST = ''
DB_PORT= ''
DB_NAME = ''
engine=create_engine(f'postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}', max_overflow=20, client_encoding='utf8')

def runQuery(sql):
    result = engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())

In [2]:
query1="""
select edbp.codenc, edbp.competencias_funcionales, edbp.numeros,
edbp.lectura_voz_alta, edbp.fluidez_lectura, edbp.cartografia, edbp.competencias_organizacionales, 
edbp.sumar, edbp.coherencia_entre_numeros, 
edbs.como_conocio_cnc, edbs.fecha_registro, 
edbs.ciudad as "ciudad_encuestador", edbs.experiencia, edbs.tipo,
edbs.barrio, edbs.fecha_nacimiento, edbs.localidad,
edbs.educacion_formal, edbs.estado_educacion, edbs.nombre_educacion
from ecar_dwh.ecar_dwh_base_psicotecnica edbp 
inner join ecar_dwh.ecar_dwh_base_sociodemografica edbs on edbs.codenc=edbp.codenc"""

query2="""
select codenc, telelink, fecha, duracion, edad,  "ciudad", "estrato", edcc.etiqueta as "genero" from (
select codenc, telelink, fecha, duracion, edad, "cod_ciudad", "ciudad", estrato as "cod_estrato", 
edcc.etiqueta as "estrato", genero from(
select codenc, telelink, fecha, duracion, edad, ciudad as "cod_ciudad", edcc.etiqueta as "ciudad", estrato, genero
from ecar_dwh.ecar_dwh_base_encuestas_efectivas edbee 
inner join ecar_dwh.ecar_dwh_base_correspondencia_codigos edcc on edbee.ciudad=edcc.codigo
where edcc.campo='ciudad')tt
inner join ecar_dwh.ecar_dwh_base_correspondencia_codigos edcc on tt.estrato=edcc.codigo
where edcc.campo='estrato')tt1
inner join ecar_dwh.ecar_dwh_base_correspondencia_codigos edcc on tt1.genero=edcc.codigo
where edcc.campo='genero'
"""

query3="""
select edbma.* from ecar_dwh.ecar_dwh_base_marcador_automatico edbma """

query4="""
select *  from ecar_dwh.ecar_dwh_base_muestra_ECAR_mensual_cuotas """

query5="""
select *  from ecar_dwh.ecar_dwh_base_fechas """


#df: Códigos left sociodemografica y psicotecnica
df=runQuery(query1)
#df2: Codigos left efectivas 
df2=runQuery(query2)
#df3: ´Marcador automático left códigos
df3=runQuery(query3)
#df4: Cuotas
df4=runQuery(query4)
#df5: fechas
df5=runQuery(query5)

In [3]:
df1=df.groupby(['codenc', 'fecha_registro']).agg({'competencias_funcionales':"mean", 'numeros':"mean", 'lectura_voz_alta':"mean",
       'fluidez_lectura':"mean", 'cartografia':"mean", 'competencias_organizacionales':"mean",
       'sumar':"mean", 'coherencia_entre_numeros':"mean", "como_conocio_cnc":"max", 
       'ciudad_encuestador':"max", 'experiencia':"max", 'tipo':"max", 'barrio':"max",
       'fecha_nacimiento':"max", 'localidad':"max", 'educacion_formal':"max", 'estado_educacion':"max",
       'nombre_educacion':"max"}).reset_index()
df1=df1[~((df1['codenc']=='Encuestador 135')&(df1['fecha_registro']=='2018-04-01'))]
df1

,codenc,fecha_registro,competencias_funcionales,numeros,lectura_voz_alta,fluidez_lectura,cartografia,competencias_organizacionales,sumar,coherencia_entre_numeros,como_conocio_cnc,ciudad_encuestador,experiencia,tipo,barrio,fecha_nacimiento,localidad,educacion_formal,estado_educacion,nombre_educacion
0,Encuestador 1,2017-05-25,3.73,4.17,4.0,4.0,3.75,4.0,5.0,2.50,internet - computrabajo.com,25175.0,0-6 meses,e,centro,1979-11-05,no_disponible,profesional,aplazado,filología inglés
1,Encuestador 10,2016-07-13,4.13,5.00,4.0,4.0,3.75,4.0,5.0,5.00,voz a voz- referido por un conocido,11001.0,no tiene,e,bosa laureles,NaN,no_disponible,no_disponible,no_disponible,no_disponible
2,Encuestador 100,2016-07-14,3.61,1.94,5.0,5.0,3.75,3.0,0.0,3.33,internet - zonajobs.com,11001.0,6 meses - 1 año,e,el ensueño-casa grande,NaN,no_disponible,no_disponible,no_disponible,no_disponible
3,Encuestador 102,2017-11-14,1.80,4.17,0.0,0.0,3.75,4.0,5.0,2.50,internet - computrabajo.com,11001.0,0-6 meses,e,san jose sur,1998-12-03,rafael uribe uribe,tecnólogo,no_disponible,tecnología en control ambiental
4,Encuestador 104,2019-05-30,1.25,4.58,0.0,0.0,1.25,5.0,5.0,5.00,voz a voz- referido por un conocido,11001.0,0-6 meses,e,restrepo,1999-09-12,antonio nariño,bachiller,culminado,no_disponible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,Encuestador 91,2017-09-29,4.26,4.58,4.0,4.0,1.25,4.0,5.0,3.75,voz a voz- referido por un conocido,11001.0,0-6 meses,e,quintanares,NaN,soacha,bachiller,culminado,no_disponible
182,Encuestador 93,2017-10-30,1.80,3.33,0.0,0.0,3.75,4.0,5.0,0.00,otros,11001.0,0-6 meses,e,divino niño,1999-07-12,bosa,bachiller,culminado,no_disponible
183,Encuestador 95,2017-02-19,4.25,5.00,4.0,4.0,2.50,4.0,5.0,5.00,otros,11001.0,0-6 meses,e,veraguas,1994-08-20,puente aranda,bachiller,culminado,no_disponible
184,Encuestador 96,2016-07-14,4.19,5.00,4.0,4.0,3.75,4.0,5.0,5.00,voz a voz- referido por un conocido,11001.0,6 meses - 1 año,e,el ensueño-casa grande,NaN,no_disponible,no_disponible,no_disponible,no_disponible


In [4]:
df2['cod_rango_edad']=1
listas=[list(range(12,18)), list(range(18,25)), list(range(25,35)),list(range(35,45)),list(range(45,55))]
listas2=[1,2,3,4,5]
for i, j in zip(listas, listas2):
    df2.loc[df2['edad'].isin(i), ['cod_rango_edad']]=j

df2=df2.merge(df4[['etiquetas_rango_edad', 'cod_rango_edad']], on='cod_rango_edad', how="left")
df2

,codenc,telelink,fecha,duracion,edad,ciudad,estrato,genero,cod_rango_edad,etiquetas_rango_edad
0,Encuestador 280,163503.0,2017-10-06,618,15,Bogotá,Medio Bajo,Hombre,1,12 a 17 años
1,Encuestador 280,163503.0,2017-10-06,618,15,Bogotá,Medio Bajo,Hombre,1,12 a 17 años
2,Encuestador 280,163503.0,2017-10-06,618,15,Bogotá,Medio Bajo,Hombre,1,12 a 17 años
3,Encuestador 280,163503.0,2017-10-06,618,15,Bogotá,Medio Bajo,Hombre,1,12 a 17 años
4,Encuestador 280,163503.0,2017-10-06,618,15,Bogotá,Medio Bajo,Hombre,1,12 a 17 años
...,...,...,...,...,...,...,...,...,...,...
2560693,Encuestador 296,155632.0,2017-08-31,558,42,Bogotá,Bajo,Mujer,4,35 a 44
2560694,Encuestador 296,155632.0,2017-08-31,558,42,Bogotá,Bajo,Mujer,4,35 a 44
2560695,Encuestador 296,155632.0,2017-08-31,558,42,Bogotá,Bajo,Mujer,4,35 a 44
2560696,Encuestador 296,155632.0,2017-08-31,558,42,Bogotá,Bajo,Mujer,4,35 a 44


In [5]:
df3['archivo']=df3['archivo'].astype(str)
df3['mes']=df3['archivo'].str.split('_',0).str[1]
df3['año']=df3['archivo'].str.split('_',0).str[2].str.split('.',0).str[0]
df3.drop('archivo', axis=1, inplace=True)

VCD = df3.drop_duplicates()
VCD['n_mes'] = VCD['mes']

List1=['espera','hablado','disponible','pausas','muerto','cliente','conectado']
for i in List1:
    VCD[i]=(VCD[i]).astype(float)

VCD['n_mes'] = VCD['n_mes'].replace({'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8, 
                     'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12})
VCD['dia']=1
lista1=list(range(1,13))
lista2=[31,28,31,30,31,30,31,31,30,31,30,31]
for i,j in zip(lista1,lista2):
    VCD.loc[VCD['n_mes']==i,['dia']]=j


VCD['fecha'] = VCD['año'].astype(str)+ ' ' +VCD['n_mes'].astype(str)+ ' ' +VCD['dia'].astype(str)
VCD['fecha'] = pd.to_datetime(VCD['fecha']) 


VCD_id = VCD.groupby(['codenc','fecha']).agg({'calls': ['count', 'sum'], 'espera': ['sum'], 'hablado': ['sum'], 'disponible': ['sum'], 
                                          'pausas': ['sum'], 'muerto': ['sum'], 'cliente': ['sum'],'conectado':['sum']}).reset_index() #, 'conectado': ['sum']
VCD_id.dtypes
VCD_id.columns=['codenc', 'fecha', 'N', 'calls', 'espera', 'hablado', 'disponible', 'pausas', 'muerto', 'cliente','conectado']

#Recalulando los porcentajes
VCD_id['total'] = VCD_id['espera'] + VCD_id['hablado'] + VCD_id['disponible'] + VCD_id['pausas']
VCD_id['porcentaje_espera'] = (VCD_id[('espera')] / VCD_id['total'] )*100
VCD_id['porcentaje_hablado'] = (VCD_id['hablado'] / VCD_id['total'] )*100
VCD_id['porcentaje_disponible'] = (VCD_id['disponible'] / VCD_id['total'])*100 
VCD_id['porcentaje_muerto'] = (VCD_id['muerto'] / VCD_id['total'] )*100
VCD_id['porcentaje_pausas'] = (VCD_id['pausas'] / VCD_id['total'] )*100
VCD_id=VCD_id[['codenc', 'calls','total', 'espera', 'porcentaje_espera', 'hablado',
       'porcentaje_hablado', 'disponible', 'porcentaje_disponible', 'pausas',
       'porcentaje_pausas', 'muerto', 'porcentaje_muerto', 'cliente',
       'conectado', 'fecha']]

In [6]:
df2['mes']= pd.DatetimeIndex(df2['fecha']).month
df2['dia']= pd.DatetimeIndex(df2['fecha']).day
df2['año']= pd.DatetimeIndex(df2['fecha']).year
df_dias_trabajados=df2.groupby(['codenc', 'año', 'mes'])[['dia']].nunique().reset_index().rename(columns={'dia':'dias_trabajados'})

df_dias_trabajados['dia']=1
lista1=list(range(1,13))
lista2=[31,28,31,30,31,30,31,31,30,31,30,31]
for i,j in zip(lista1,lista2):
    df_dias_trabajados.loc[df_dias_trabajados['mes']==i,['dia']]=j
df_dias_trabajados['fecha']= df_dias_trabajados['año'].astype(str)+ ' ' +df_dias_trabajados['mes'].astype(str)+ ' ' +df_dias_trabajados['dia'].astype(str)
df_dias_trabajados['fecha']=pd.to_datetime(df_dias_trabajados['fecha']) 
df_dias_trabajados

,codenc,año,mes,dias_trabajados,dia,fecha
0,Encuestador 1,2017,6,3,30,2017-06-30
1,Encuestador 1,2017,7,17,31,2017-07-31
2,Encuestador 10,2018,4,2,30,2018-04-30
3,Encuestador 10,2018,5,4,31,2018-05-31
4,Encuestador 100,2017,3,20,31,2017-03-31
...,...,...,...,...,...,...
1497,Encuestador 99,2018,1,24,31,2018-01-31
1498,Encuestador 99,2018,2,22,28,2018-02-28
1499,Encuestador 99,2018,3,24,31,2018-03-31
1500,Encuestador 99,2018,4,25,30,2018-04-30


In [7]:
df2['fecha']=pd.to_datetime(df2['fecha'])
efectivas_agg=df2.groupby([pd.Grouper(key='fecha',freq='M'),'codenc']).agg({'duracion':'sum','genero':'count'}).reset_index().rename(columns={'genero':'numero efectivas'})
VCD_efectivas=VCD_id.merge(efectivas_agg,on=['codenc','fecha'],how='left')
VCD_efectivas['porcentaje_efectivas']=(VCD_efectivas['numero efectivas']/VCD_efectivas['calls'])*100
VCD_efectivas=VCD_efectivas.merge(df_dias_trabajados[['codenc', 'fecha', 'dias_trabajados']], on=['codenc','fecha'],how='left')
VCD_efectivas['efectividad_dia']=(VCD_efectivas['porcentaje_efectivas']/VCD_efectivas['dias_trabajados'])
VCD_efectivas

,codenc,calls,total,espera,porcentaje_espera,hablado,porcentaje_hablado,disponible,porcentaje_disponible,pausas,...,muerto,porcentaje_muerto,cliente,conectado,fecha,duracion,numero efectivas,porcentaje_efectivas,dias_trabajados,efectividad_dia
0,Encuestador 1,4429,623484.0,150572.0,24.150098,313690.0,50.312438,42598.0,6.832252,116624.0,...,2193.0,0.351733,311497.0,2281.0,2017-07-31,1408770.0,1278.0,28.855272,17.0,1.697369
1,Encuestador 10,647,72817.0,18555.0,25.481687,31013.0,42.590329,9059.0,12.440776,14190.0,...,171.0,0.234835,30842.0,406.0,2018-04-30,63756.0,72.0,11.128284,2.0,5.564142
2,Encuestador 10,2369,222791.0,63164.0,28.351235,91214.0,40.941510,20844.0,9.355854,47569.0,...,407.0,0.182682,90807.0,1862.0,2018-05-31,257688.0,252.0,10.637400,4.0,2.659350
3,Encuestador 100,6077,818744.0,218264.0,26.658394,352053.0,42.999155,102520.0,12.521618,145907.0,...,2567.0,0.313529,349486.0,3165.0,2017-07-31,2252628.0,2862.0,47.095606,23.0,2.047635
4,Encuestador 100,5572,806027.0,216808.0,26.898355,340585.0,42.254788,109026.0,13.526346,139608.0,...,4221.0,0.523680,336364.0,3012.0,2017-08-31,2231136.0,3006.0,53.948313,22.0,2.452196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380,Encuestador 99,6942,927154.0,254435.0,27.442582,443694.0,47.855480,60061.0,6.477996,168964.0,...,13361.0,1.441077,430333.0,7763.0,2018-01-31,2409876.0,2970.0,42.783060,24.0,1.782627
1381,Encuestador 99,6488,881291.0,224231.0,25.443469,432974.0,49.129516,63884.0,7.248911,160202.0,...,7350.0,0.834004,425624.0,8909.0,2018-02-28,2325726.0,2736.0,42.170160,22.0,1.916825
1382,Encuestador 99,6949,914133.0,262009.0,28.662022,424128.0,46.396750,61194.0,6.694212,166802.0,...,3997.0,0.437245,420131.0,5136.0,2018-03-31,2081826.0,2700.0,38.854511,24.0,1.618938
1383,Encuestador 99,7597,921118.0,252071.0,27.365766,438330.0,47.586737,57417.0,6.233403,173300.0,...,4639.0,0.503627,433691.0,6495.0,2018-04-30,2145042.0,2610.0,34.355667,25.0,1.374227


In [8]:
def count_var(df2, var):
    data=df2[['fecha', 'codenc']].drop_duplicates()
    for i in df2[var].unique(): 
        dfs=df2[df2[var]==i].groupby([pd.Grouper(key='fecha',freq='M'),'codenc']).agg({'telelink':'count'}).reset_index().rename(columns={'telelink':'efectivas '+str(i)})
        data=dfs.merge(data, on=['fecha', 'codenc'], how='outer')
    return data

df2_genero=count_var(df2, 'genero')
df2_estrato=count_var(df2, 'estrato')
df2_ciudad=count_var(df2, 'ciudad')
df2_edad=count_var(df2, 'cod_rango_edad')

In [9]:
dt=VCD_efectivas.merge(df2_genero, on=['codenc','fecha'],how='left')
dt=dt.merge(df2_estrato, on=['codenc','fecha'],how='left')
dt=dt.merge(df2_ciudad, on=['codenc','fecha'],how='left')
marcador_efectivas=dt.merge(df2_edad,  on=['codenc','fecha'],how='left')
marcador_efectivas=marcador_efectivas.fillna(0)
#marcador_efectivas
dt=marcador_efectivas.pivot(index='codenc',columns='fecha',values='calls').reset_index()
meses_trabajados=pd.DataFrame(dt.groupby('codenc').count().sum(axis=1)).reset_index()
meses_trabajados.columns=['codenc', 'meses_trabajados']
meses_trabajados=meses_trabajados.sort_values('meses_trabajados', ascending=False)

In [10]:
cols=['codenc','calls', 'total', 'espera', 'hablado', 'disponible', 'pausas', 'muerto', 'duracion', 
      'numero efectivas', 'dias_trabajados', 'efectivas Mujer', 'efectivas Hombre', 'efectivas  Alto',
       'efectivas  Medio Alto', 'efectivas  Medio Medio', 'efectivas  Bajo',
       'efectivas  Medio Bajo', 'efectivas Tunja', 'efectivas Popayán',
       'efectivas Montería', 'efectivas Manizales', 'efectivas Armenia',
       'efectivas Villavicencio', 'efectivas Pasto', 'efectivas Cúcuta',
       'efectivas Santa Marta', 'efectivas Ibagué', 'efectivas Neiva',
       'efectivas Pereira', 'efectivas Cartagena', 'efectivas Cali',
       'efectivas Bucaramanga', 'efectivas Barranquilla', 'efectivas Medellín',
       'efectivas Bogotá', 'efectivas 5', 'efectivas 2', 'efectivas 4',
       'efectivas 3', 'efectivas 1']
marcador_efectivas_total=marcador_efectivas.groupby('codenc')[cols].mean().reset_index().rename(columns={'0': 'mean '+ str(i)})
marcador_efectivas_total=marcador_efectivas_total.merge(meses_trabajados, on='codenc', how='left')
df_total=marcador_efectivas_total.merge(df1, on='codenc', how='left')
df_total_codenc_comun=marcador_efectivas_total.merge(df1, on='codenc', how='inner')

(172, 61)

In [11]:
df_total.columns=['codenc','calls', 'total', 'espera', 'hablado', 'disponible', 'pausas', 'muerto', 'duracion_efectivas', 
      'numero_efectivas', 'dias_trabajados', 'efectivas_Mujer', 'efectivas_Hombre', 'efectivas_Alto',
       'efectivas_Medio_Alto', 'efectivas_Medio_Medio', 'efectivas_Bajo',
       'efectivas_Medio_Bajo', 'efectivas_Tunja', 'efectivas_Popayán',
       'efectivas_Montería', 'efectivas_Manizales', 'efectivas_Armenia',
       'efectivas_Villavicencio','efectivas_Pasto', 'efectivas_Cúcuta',
       'efectivas_Santa Marta', 'efectivas_Ibagué', 'efectivas_Neiva',
       'efectivas_Pereira', 'efectivas_Cartagena', 'efectivas_Cali',
       'efectivas_Bucaramanga','efectivas_Barranquilla', 'efectivas_Medellín',
       'efectivas_Bogotá', 'efectivas_rango_edad_5', 'efectivas_rango_edad_2',
      'efectivas_rango_edad_4', 'efectivas_rango_edad_3', 'efectivas_rango_edad_1',
        'meses_trabajados', 'fecha_registro', 'competencias_funcionales', 'numeros', 
      'lectura_voz_alta','fluidez_lectura', 'cartografia', 'competencias_organizacionales',
       'sumar', 'coherencia_entre_numeros', 'como_conocio_cnc', 'ciudad_encuestador', 'experiencia', 'tipo', 'barrio',
       'fecha_nacimiento', 'localidad', 'educacion_formal', 'estado_educacion','nombre_educacion']
      
df_total.to_csv('data_etls/ecar_dwh_base_modelo.csv', index=False)

In [13]:
df_total

,codenc,calls,total,espera,hablado,disponible,pausas,muerto,duracion_efectivas,numero_efectivas,...,como_conocio_cnc,ciudad_encuestador,experiencia,tipo,barrio,fecha_nacimiento,localidad,educacion_formal,estado_educacion,nombre_educacion
0,Encuestador 1,4429.000000,623484.000000,150572.000000,313690.000000,42598.000000,116624.000000,2193.000000,1.408770e+06,1278.000000,...,internet - computrabajo.com,25175.0,0-6 meses,e,centro,1979-11-05,no_disponible,profesional,aplazado,filología inglés
1,Encuestador 10,1508.000000,147804.000000,40859.500000,61113.500000,14951.500000,30879.500000,289.000000,1.607220e+05,162.000000,...,voz a voz- referido por un conocido,11001.0,no tiene,e,bosa laureles,NaN,no_disponible,no_disponible,no_disponible,no_disponible
2,Encuestador 100,5743.000000,820764.181818,235728.090909,330865.454545,88244.272727,165926.363636,7262.818182,1.917841e+06,2613.272727,...,internet - zonajobs.com,11001.0,6 meses - 1 año,e,el ensueño-casa grande,NaN,no_disponible,no_disponible,no_disponible,no_disponible
3,Encuestador 101,626.500000,125574.500000,37047.000000,49981.500000,9136.000000,29410.000000,3586.500000,3.012840e+05,378.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Encuestador 102,3744.500000,510847.500000,180869.500000,202350.000000,30753.500000,96874.500000,5939.500000,7.392960e+05,972.000000,...,internet - computrabajo.com,11001.0,0-6 meses,e,san jose sur,1998-12-03,rafael uribe uribe,tecnólogo,no_disponible,tecnología en control ambiental
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,Encuestador 95,5890.666667,760716.000000,226663.000000,311614.333333,82926.000000,139512.666667,4070.333333,1.862220e+06,2580.000000,...,otros,11001.0,0-6 meses,e,veraguas,1994-08-20,puente aranda,bachiller,culminado,no_disponible
280,Encuestador 96,3484.800000,501703.800000,156219.600000,174043.800000,79735.400000,91705.000000,5142.000000,1.142222e+06,1414.800000,...,voz a voz- referido por un conocido,11001.0,6 meses - 1 año,e,el ensueño-casa grande,NaN,no_disponible,no_disponible,no_disponible,no_disponible
281,Encuestador 97,3227.333333,554952.000000,112426.666667,252206.666667,69592.000000,120726.666667,4709.000000,1.611480e+06,1632.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
282,Encuestador 98,3591.000000,604918.000000,134926.000000,305491.000000,40198.000000,124303.000000,2648.000000,1.680912e+06,1692.000000,...,internet - computrabajo.com,11001.0,0-6 meses,e,republica de venezuela,1996-06-14,ciudad bolivar,tecnólogo,culminado,tecnología de gestión integrada
